# Introduction

Atlanta, my hometown, is a highly diverse city with poorly-defined neighborhoods. While the neighborhoods are often defined by road boundaries, they are characterized by a number of other factors. In this project, I will use data from Four Square to examine the fluctuating popularity of restaurants of various kinds and use that information to classify locations into neighborhoods. I will then compare that data with google-assigned neighborhoods and determine if the two classifications are similar. 

# Data

This project will utilize data from 2 sources:
1. Trending restaurant data from Foursquare. This will be pulled between 6-7 PM on a Friday (peak restaurant selection time) and saved to a file, since repeated calls to the Foursquare API would result in different trending information. An extension of this project could be to examine the fluctuation of neighborhood boundaries over the course of a week, but that is beyond the scope of this project. 
2. Neighborhood data from Wikipedia, specifically this article: https://en.wikipedia.org/wiki/Neighborhoods_in_Atlanta These are the "official" neighborhoods 

# Initializations

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import folium

# Collect Atlanta Neighborhood Geographical Data

# Connect with Foursquare

# Clustering Analysis